### import bibliotek

In [33]:
import os
import json
import requests
from bs4 import BeautifulSoup

### Narzędzia

In [34]:
def extract(ancestor, selector=None, attribute=None, many=False):
    if selector:
        if many:
            if attribute:
                return [item[attribute].strip() for item in ancestor.select(selector)]
            return [item.text.strip() for item in ancestor.select(selector)]     
        if attribute:
            try:
                return ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None
        try:
            return ancestor.select_one(selector).text.strip()
        except AttributeError:
            return None
    if attribute:
        try: 
            return ancestor[attribute].strip()
        except TypeError:
            return
    return ancestor.text.strip()



In [36]:
review_schema = {
    "review_id": ( None, 'data-entry-id'),
    "author": ("span.user-post__author-name",),
    "recomendation": ("span.user-post__author-recomendation > em",),
    "stars": ("span.user-post__score-count",),
    "content": ("div.user-post__text",),
    "pros": ("div.review-feature__item--positive",None, True),
    "cons": ("div.review-feature__item--negative", None, True),
    "likes": ("button.vote-yes > span",),
    "dislikes": ("button.vote-no > span",),
    "publish_date": ("span.user-post__published > time:nth-child(1)",'datetime'),
    "purchase_date": ("span.user-post__published > time:nth-child(2)",'datetime'),
                    }

### Pobranie ze strony Ceneo.pl opinii o konkretnym produkcie

In [37]:
with open("./headers.json", "r") as jf:
    headers = json.load(jf)

In [42]:
product_id = "16524500"
next_page = f"https://www.ceneo.pl/{product_id}#tab=reviews"
all_reviews = []
while next_page:
    response = requests.get(next_page, headers=headers)
    print(next_page)
    if response.status_code == 200:
        page_dom = BeautifulSoup(response.text, "html.parser")
        reviews = page_dom.select("div.js_product-review:not(.user-post--highlight)")
        print(len(reviews))
        for review in reviews:
                single_review = {
                    key: extract(review, *value)
                    for key, value in review_schema.items()
                }
                all_reviews.append(single_review)
        try:
            next_page = "https://www.ceneo.pl" + page_dom.select_one("a.pagination__next")["href"]
        except TypeError:
            next_page = None    

https://www.ceneo.pl/16524500#tab=reviews
10
https://www.ceneo.pl/16524500/opinie-2
0


### Zapisanie wszystkich opinii o konkretnym produkcie w bazie danych

In [39]:
if not os.path.exists("./opinions"):
    os.mkdir("./opinions")

In [32]:
with open(f"./opinions/{product_id}.json", "w", encoding="UTF-8") as jf:
    json.dump(all_reviews, jf, indent=4, ensure_ascii=False)